###  Pasos a seguir en el proceso de 'scraping':

1. Encuentra la URL que quieres 'escrapear'.
2. Inspecciona la página (código fuente).
3. Localiza los datos que necesitas obtener.
4. Desarrolla tu código en Python.
    1. Crea tu sopa
    2. Busca los elementos que cotienen los datos y extráelos
5. Ejecuta tu código y obten los datos.
6. Almacena los datos en el formato requerido.

In [1]:
# Importamos librerías
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

## Caso 1: Scraping de un catálogo: Labirratorium

In [2]:
URL = "https://www.labirratorium.com/es/67-cervezas-por-estilo?page="

Queremos obtener un dataFrame con todas las cervezas del catálogo y sus características descritas. Analizamos la página para ver qué tenemos que hacer para conseguirlo

In [3]:
# La web tiene 49 páginas con 12 cervezas listadas en cada página.

Hacemos la consulta (request) y creamos la SOPA inicial:

In [9]:
web_html = requests.get(URL).content
soup = bs(web_html, "lxml")
type(soup)

bs4.BeautifulSoup

In [31]:
ls_href_tags[0]["href"]

'https://www.labirratorium.com/es/inicio/831-arriaca-boris-brew-vikingathor-.html'

In [34]:
ls_href_tags = soup.find_all("a", class_="thumbnail product-thumbnail")
ls_hrefs = [tag["href"] for tag in ls_href_tags]
# ls_hrefs = []
# for tag in ls_href_tags:
#     ls_hrefs.append(tag["href"])

ls_hrefs


['https://www.labirratorium.com/es/inicio/831-arriaca-boris-brew-vikingathor-.html',
 'https://www.labirratorium.com/es/cerveza-lambic/284-boon-kriek-2013.html',
 'https://www.labirratorium.com/es/cervezas-alemanas/225-stortebeker-schwarz-bier.html',
 'https://www.labirratorium.com/es/botella-33cl/199-orval.html',
 'https://www.labirratorium.com/es/cervezas-alemanas/184-augustiner-lagerbier-hell.html',
 'https://www.labirratorium.com/es/inicio/183-schneider-eisbock.html',
 'https://www.labirratorium.com/es/inicio/181-schlenkerla-rauchbier-weizen.html',
 'https://www.labirratorium.com/es/inicio/173-samuel-adams-boston-lager.html',
 'https://www.labirratorium.com/es/inicio/82-westmalle-dubbel.html',
 'https://www.labirratorium.com/es/inicio/75-duchesse-de-bourgogne.html',
 'https://www.labirratorium.com/es/inicio/61-tripel-karmeliet33.html',
 'https://www.labirratorium.com/es/inicio/21-weihenstephaner-vitus.html']

Nombre,	precio,	descripcion corta, descripción larga, ruta imagen, marca, barcode,	Estilo, Origen, % Alc., % Alc. Exacto, Volumen (Cl), Tipo Fermentación, IBU, IBU Exacto, Color, Envase

In [38]:
first_url = ls_hrefs[0]
first_data = requests.get(first_url).content
first_soup = bs(first_data, "lxml")

In [67]:
first_name = first_soup.find("h1", class_="h1 product-detail-name").string
print(first_name)

div_main_info = first_soup.find("div", class_= "col-form_id-form_15874367062488778 col-md-6 col-lg-6 col-xl-6")
first_price = div_main_info.find("span", itemprop="price")["content"]
print(first_price)

first_short_description = div_main_info.find("div", itemprop="description").p.string
print(first_short_description)

div_img_cover = first_soup.find("div", class_="product-cover")
first_img = div_img_cover.find("img", itemprop="image")["src"]
print(first_img)

first_long_description = first_soup.find("div", class_="product-description").text
print(first_long_description)

data_sheet = first_soup.find("dl", class_="data-sheet")
# first_estilo=

ls_names = [tag.string for tag in data_sheet.find_all("dt")]
ls_values = [tag.string for tag in data_sheet.find_all("dd")]

first_d_props = dict(zip(ls_names, ls_values))

first_d_total = {"Nombre": first_name, 
                 "Precio": first_price, 
                 "Desc Corta": first_short_description,
                 "Imagen": first_img,
                 "Desc Larga": first_long_description}

first_d_total.update(first_d_props)

first_d_total


SanFrutos Vikingathor Boris Brew
3.35
Indian Dunkel
https://www.labirratorium.com/27941-large_default/arriaca-boris-brew-vikingathor-.jpg
Cerveza de 8.2% ABV intensa, oscura y muy lupulada. Con aroma intenso a maltas tostadas, caramelo, fruta madura, resina e hierbas aromáticas. Delicioso armargor, dulzor equilibrado y sensación de calidez. Refrescante y lupulada. 
Una receta de Boris de Mesones.
Vikingathor ya ha sido galardonada con anterioridad con las Medallas de Plata cosechadas en el European Beer Star de 2011 y 2012 y la Medalla de Plata en el Asia Beer Cup 2013.




{'Nombre': 'SanFrutos Vikingathor Boris Brew',
 'Precio': '3.35',
 'Desc Corta': 'Indian Dunkel',
 'Imagen': 'https://www.labirratorium.com/27941-large_default/arriaca-boris-brew-vikingathor-.jpg',
 'Desc Larga': 'Cerveza de 8.2% ABV intensa, oscura y muy lupulada. Con aroma intenso a maltas tostadas, caramelo, fruta madura, resina e hierbas aromáticas. Delicioso armargor, dulzor equilibrado y sensación de calidez. Refrescante y lupulada.\xa0\nUna receta de Boris de Mesones.\nVikingathor ya ha sido galardonada con anterioridad con las\xa0Medallas de Plata\xa0cosechadas en el\xa0European Beer Star\xa0de 2011 y 2012 y la\xa0Medalla de Plata\xa0en el\xa0Asia Beer Cup\xa02013.\n\n',
 'Estilo': 'DUNKEL',
 'Origen': 'Segovia',
 '% Alc.': 'ALTO (6-9%)',
 '% Alc. Exacto': '8.2',
 'Volumen (cl)': '33 Cl',
 'Tipo Fermentación': 'Lager (Baja Fermentación)',
 'IBU': '50-75 Amargor alto',
 'IBU Exacto': '75',
 'Color': 'Marrón',
 'Envase': 'Botella'}

'3.35'

Ya tenemos todos los datos que queremos de la cerveza: Agrupamos todo en una lista:

### Ya sabemos obtener todos los datos que nos interesan de una cerveza, ahora tenemos que aplicar esta lógica para obtener todas las demás 

Construir un DataFrame con 

Nombre,	precio,	descripcion corta, descripción larga, ruta imagen, marca, barcode,	Estilo, Origen, % Alc., % Alc. Exacto, Volumen (Cl), Tipo Fermentación, IBU, IBU Exacto, Color, Envase